In [1]:
import pickle
import os
import pandas as pd
from tqdm import tqdm
from src.models import *
from src.constants import *
from src.plotting import *
from src.pot import *
from src.utils import *
from src.diagnosis import *
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn
from time import time
from pprint import pprint
from datetime import datetime, timedelta
from main import  load_dataset, backprop
import matplotlib.pyplot as plt
import random

%matplotlib inline

plt.rcParams["text.usetex"] = False
plt.rcParams['figure.figsize'] = 8, 5

def normalize3(a, min_a=None, max_a=None):
    if min_a is None: min_a, max_a = np.min(a, axis=0), np.max(a, axis=0)
    return ((a - min_a) / (max_a - min_a + 0.0001)), min_a, max_a

def convert_to_windows(data, model):
    windows = []
    w_size = model.n_window
    for i, g in enumerate(data):
        if i >= w_size:
            w = data[i - w_size:i]  # cut
        else:
            w = torch.cat([data[0].repeat(w_size - i, 1), data[0:i]])  # pad
        windows.append(w if 'DTAAD' in model.name or 'Attention' in model.name or 'TranAD' in model.name else w.view(-1))
    return torch.stack(windows)

def load_model(modelname, dims):
    import src.models
    model_class = getattr(src.models, modelname)
    model = model_class(dims).double()
    fname = f'checkpoints/{modelname}_{args.dataset}/model.ckpt'
    if os.path.exists(fname) and (not args.retrain or args.test):
        checkpoint = torch.load(fname, weights_only=False)
        model.load_state_dict(checkpoint['model_state_dict'])
    else:
        print(f"{color.GREEN}Creating new model: {model.name}{color.ENDC}")
        assert True
    return model

def filter_noise_es(df, alpha=0.4, reduction=False):
    import copy
    new_df = copy.deepcopy(df)
    
    for column in df:
        new_df[column] = df[column].ewm(alpha=alpha, adjust=False).mean()
    
    if reduction:
        return new_df[::len(df)]  # Adjust sparsity if needed
    else:
        return new_df

def wgn_pandas(df_withtime, snr, alpha=0.15, window_size=120):
    df_no_timestamp = df_withtime.drop(columns=['TimeStamp'])
    noisy_df = pd.DataFrame(index=df_no_timestamp.index, columns=df_no_timestamp.columns)

    for start in range(0, len(df_no_timestamp), window_size):
        window = df_no_timestamp.iloc[start:start + window_size]
        
        min_window, max_window = window.min(), window.max()
        #x = (window - min_window) / (max_window - min_window + 1e-4)
        Ps = np.sum(np.power(window, 2), axis=0) / len(window)
        Pn = Ps / (np.power(10, snr / 10))

        noise = np.random.randn(*window.shape) * np.sqrt(Pn.values)
        noisy_window = window + (noise / 100)

        noisy_df.iloc[start:start + window_size] = noisy_window
    
    noisy_df.reset_index(drop=True, inplace=True)
    noisy_df = filter_noise_es(pd.DataFrame(noisy_df, columns=noisy_df.columns), alpha)

    df_timestamp = df_withtime['TimeStamp']
    df_timestamp.reset_index(drop=True, inplace=True)

    df_withtime = pd.concat([df_timestamp, noisy_df], axis=1)
    return df_withtime

feature_set = ['Active Power', 'Reactive Power', 'Governor speed actual', 'UGB X displacement', 'UGB Y displacement',
    'LGB X displacement', 'LGB Y displacement', 'TGB X displacement',
    'TGB Y displacement', 'Stator winding temperature 13',
    'Stator winding temperature 14', 'Stator winding temperature 15',
    'Surface Air Cooler Air Outlet Temperature',
    'Surface Air Cooler Water Inlet Temperature',
    'Surface Air Cooler Water Outlet Temperature',
    'Stator core temperature', 'UGB metal temperature',
    'LGB metal temperature 1', 'LGB metal temperature 2',
    'LGB oil temperature', 'Penstock Flow', 'Turbine flow',
    'UGB cooling water flow', 'LGB cooling water flow',
    'Generator cooling water flow', 'Governor Penstock Pressure',
    'Penstock pressure', 'Opening Wicked Gate', 'UGB Oil Contaminant',
    'Gen Thrust Bearing Oil Contaminant']

dataset_folder = 'data/CustomAWGN30ES15'
df_train = pd.read_csv(os.path.join(dataset_folder, 'train.csv'))
df_test = pd.read_csv(os.path.join(dataset_folder, 'test.csv'))
df_train, df_test = df_train.values[:, 1:], df_test.values[:, 1:]
_, min_a, max_a = normalize3(np.concatenate((df_train, df_test), axis=0))

model_array = ["Attention", "DTAAD", "MAD_GAN", "TranAD", "DAGMM", "USAD"] # , CAE_M "GDN" MSCRED
model_thr = {"Attention": 0, "DTAAD": 0, "MAD_GAN": 0, "TranAD": 0, "DAGMM": 0, "USAD": 0}

for model_now in model_array:
    with open(f'loss_fold/{model_now}.pickle', 'rb') as handle:
        loss = pickle.load(handle)
    model_thr[model_now] = [np.percentile(loss[:, index], 99) for index in range(len(feature_set))]

measured_horizon = 60 * 3 * 1

df_data_withtime = pd.read_pickle("/run/media/fourier/Data2/Pras/Vale/time-series-autoencoder/my_data_5thn_olah.pickle")
mask = (df_data_withtime['TimeStamp'] >= '2020-01-01 00:00:00')
df_data_withtime = df_data_withtime.loc[mask]

for column_name in df_data_withtime.columns:
    if column_name != 'Load_Type' and column_name != 'TimeStamp':
        df_data_withtime[column_name] = pd.to_numeric(df_data_withtime[column_name], downcast='float')
        
df_anomaly = pd.read_excel("/run/media/fourier/Data2/Pras/Vale/time-series-autoencoder/shutdown_list.xlsx", 'Sheet2')
df_anomaly['Start Time'] = pd.to_datetime(df_anomaly['Start Time'])
df_anomaly['End Time'] = pd.to_datetime(df_anomaly['End Time'])
df_anomaly_unplaned = df_anomaly.copy()

mask = (df_anomaly_unplaned['Interal/External'] == 'Internal') & (df_anomaly_unplaned['Shutdown Type'] == 'Unplanned') & (df_anomaly_unplaned['Start Time'] >= '2020-01-01 00:00:00')
df_anomaly_unplaned = df_anomaly_unplaned.loc[mask]
df_anomaly_unplaned = df_anomaly_unplaned.reset_index(drop=True)
df_anomaly_unplaned = df_anomaly_unplaned.drop(df_anomaly_unplaned.index[[2]])
df_anomaly_unplaned

,Start Time,End Time,Event,Activities,Related Component,Action/Cause,Interal/External,Shutdown Details,Startup Details,Supplied Grid after Action,Shutdown Type
0,2023-05-22 20:13:00,2023-05-22 20:32:00,LGS#1 Trip by 86N energized,Mechanical,TGB Oil Level,Repairment,Internal,Trip by 86 N Alarm,Flying Started,FCE Grid,Unplanned
1,2022-03-26 10:07:00,2022-03-26 13:50:00,LGS#1 Trip by 86N energized,Mechanical,Rotor Shaft Current at Lower Generator,Repairment,Internal,Trip by 86N Alarm to Standstill,Start via 4/CS,FCE Grid,Unplanned
3,2021-05-28 06:15:00,2021-05-30 00:58:00,LGS#1 Trip by 86N energized,Mechanical,TGB Cooling Water,Repairment,Internal,Trip by 86N Alarm,Started via 4/CS,FCE Grid,Unplanned


In [2]:
failure_index_list = 1
index_before = 0

threshold_percentage_all = {}
end_date_filter = df_anomaly_unplaned.values[failure_index_list, 0] - timedelta(minutes=(100 * index_before) + 5)
start_date_filter =  end_date_filter - timedelta(minutes=measured_horizon)

mask = (df_data_withtime['TimeStamp'] > start_date_filter.strftime('%Y-%m-%d %H:%M:%S')) & (df_data_withtime['TimeStamp'] <= end_date_filter.strftime('%Y-%m-%d %H:%M:%S'))
df_sel = df_data_withtime.loc[mask]
df_sel = df_sel.reset_index(drop=True)
df_sel = wgn_pandas(df_sel, 30, alpha=0.15)

df_timestamp = df_sel.iloc[:, 0]
df_feature =  df_sel.iloc[:, 1:]
df_feature = df_feature[feature_set]
raw_active = df_feature['Active Power'].values

df_feature, _, _ = normalize3(df_feature, min_a, max_a)
df_feature = df_feature.astype(float)

test_loader = DataLoader(df_feature.values, batch_size=df_feature.shape[0])
testD = next(iter(test_loader))
testO = testD

for idx_model, model_now in enumerate(model_array):
    model = load_model(model_now, testO.shape[1])
    model.eval()
    torch.zero_grad = True

    if model.name in ['Attention', 'DAGMM', 'USAD', 'MSCRED', 
                        'CAE_M', 'GDN', 'MTAD_GAT', 'MAD_GAN', 'TranAD'] or 'DTAAD' in model.name:
        testD_now = convert_to_windows(testD, model)

    loss, y_pred = backprop(0, model, testD_now, testO, None, None, training=False)
    if 'TranAD' or 'DTAAD' in model.name: testO_now = torch.roll(testO, 1, 0)

    threshold_pass = {}
    for idx_feat in range(loss.shape[-1]):
        thres_bool = loss[:, idx_feat] > model_thr[model_now][idx_feat]
        threshold_pass[feature_set[idx_feat]] = (thres_bool.sum() / thres_bool.shape[0]) * 100
    
    threshold_pass = dict(sorted(threshold_pass.items(), key=lambda item: item[1], reverse=True)[:5])
    threshold_percentage_all[idx_model] = threshold_pass

/run/media/fourier/Data2/Pras/Vale/.env/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/run/media/fourier/Data2/Pras/Vale/.env/lib/python3.8/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [15]:
counter_feature = {}
for modex_idx, values_pred in threshold_percentage_all.items():
    for name_feat, percentage in values_pred.items():
        if name_feat in counter_feature:
            counter_feature[name_feat]["count"] = counter_feature[name_feat]["count"] + 1
            counter_feature[name_feat]["percentage"] = counter_feature[name_feat]["percentage"] + percentage
        else:
            counter_feature[name_feat] = {"count": 1, "percentage": percentage}

counter_feature_s1 = dict(sorted(counter_feature.items(), key=lambda item: item[1]['count'], reverse=True)[:4])
counter_feature_s2 = dict(sorted(counter_feature_s1.items(), key=lambda item: item[1]['percentage'], reverse=True))
counter_feature_s2

{'UGB X displacement': {'count': 4, 'percentage': 394.44444444444446},
 'LGB X displacement': {'count': 5, 'percentage': 390.0},
 'LGB Y displacement': {'count': 4, 'percentage': 350.0},
 'UGB Y displacement': {'count': 4, 'percentage': 116.66666666666667}}

In [ ]:
failure_index_list = 1
index_before = 0

thr_array_fault = {}
for failure_index_list in range(3):
    threshold_percentage_all = {}
    end_date_filter = df_anomaly_unplaned.values[failure_index_list, 0] - timedelta(minutes=(100 * index_before) + 5)
    start_date_filter =  end_date_filter - timedelta(minutes=measured_horizon)

    mask = (df_data_withtime['TimeStamp'] > start_date_filter.strftime('%Y-%m-%d %H:%M:%S')) & (df_data_withtime['TimeStamp'] <= end_date_filter.strftime('%Y-%m-%d %H:%M:%S'))
    df_sel = df_data_withtime.loc[mask]
    df_sel = df_sel.reset_index(drop=True)
    df_sel = wgn_pandas(df_sel, 30, alpha=0.15)

    df_timestamp = df_sel.iloc[:, 0]
    df_feature =  df_sel.iloc[:, 1:]
    df_feature = df_feature[feature_set]
    raw_active = df_feature['Active Power'].values

    df_feature, _, _ = normalize3(df_feature, min_a, max_a)
    df_feature = df_feature.astype(float)

    test_loader = DataLoader(df_feature.values, batch_size=df_feature.shape[0])
    testD = next(iter(test_loader))
    testO = testD

    feature_num = 31

    for idx_model, model_now in enumerate(model_array):
        model, _, _, _, _ = load_model(model_now, testO.shape[1])
        torch.zero_grad = True
        model.eval()

        if model.name in ['Attention', 'DAGMM', 'USAD', 'MSCRED', 'CAE_M', 'GDN', 'MTAD_GAT', 'MAD_GAN', 'TranAD'] or 'DTAAD' in model.name:
            testD_now = convert_to_windows(testD, model)

        loss, y_pred = backprop(0, model, testD_now, testO, None, None, training=False)
        if 'TranAD' or 'DTAAD' in model.name: testO_now = torch.roll(testO, 1, 0)

        threshold_pass = {}
        for i in range(loss.shape[-1]):
            index_plot = i + 1 + (idx_model * feature_num)
            thres_bool = loss[:, i] > model_thr[model_now][i]
            threshold_pass[feature_set[i]] = (thres_bool.sum() / thres_bool.shape[0]) * 100
        
        threshold_pass = dict(sorted(threshold_pass.items(), key=lambda item: item[1], reverse=True)[:5])
        threshold_percentage_all[model_now] = threshold_pass

    thr_array_fault[df_anomaly_unplaned.values[failure_index_list, 4]] = threshold_percentage_all